<a href="https://colab.research.google.com/github/jcj402-sys/quantum_algorithm/blob/main/shor_factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Shor's Factorization Algorithm
---

이 노트북에서는 Shor's Factorization Algorithm을 구현하는 실습을 진행할 것입니다. Basic Quantum operator의 리뷰부터 시작해서, 15의 소인수분해를 진행해보도록 하겠습니다.

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
import numpy as np
import math
import matplotlib.pyplot as plt
from fractions import Fraction
from collections import Counter

In [ ]:
#문제 1. 15의 약수 찾기 정답: 3, 5

In [ ]:
def c_amod15(a:int, power int): #3,4,5,6 과정에서 답이 아니라는 것을 알림
  if a not in [2,4,7,8,11,13]:
    raise ValueError(f"a를 다시 골라주세요")
  U=QuantumCircuit(4,name=f"{a}^{power} mod 15}")
  for in range(power):
    if a in [2,13]:
      U.swap(0,1) #0, 1 state change
      U.swap(1,2)
      U.swap(2,3)
    if a in[7,8]:
      U.swap(2,3)
      U.swap(1,2)
      U.swap(0,1)
    if a in[4,11]:
      U.swap(1,3)
      U.swap(0,2)
    if a in[7,11,13]:
      for q in range(4): U.x(q)
return u.to_gate().control()


